dataset:
http://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

In [21]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors

In [22]:
class Housing:
    def __init__(self):
        self.housing = pd.read_csv('new_housing.csv',sep=',',decimal='.',encoding='utf-8')

        
    def encode_fit_transform(self, database):
        
        # one hot encoder for ocean_proximity
        self.onehot = OneHotEncoder(sparse=False, categories= 'auto')
        ocean_labels=database['Ocean proximity'].values.reshape(-1, 1)
        onehot_ocean_labels = self.onehot.fit_transform(ocean_labels)
        
        # concatenate onehot encoded values to dataframe
        dfOneHot = pd.DataFrame(onehot_ocean_labels,
                                columns = ["Ocean_"+str(int(i)) for i in range(onehot_ocean_labels.shape[1])])
        encoded_database = pd.concat([database, dfOneHot], axis=1)
        
        #clean dataframe
        encoded_database.drop('Ocean proximity', axis = 1, inplace = True) #drop ocean_proximity original column
        encoded_database.dropna(axis=0, how='any',inplace = True) #drop NaN
        
        #dataframe columns
        self.columns = encoded_database.columns.tolist()
        
        #normilize values all columns
        self.scaler = MinMaxScaler()
        encoded_database[self.columns] = self.scaler.fit_transform(encoded_database[self.columns])
        return encoded_database
    
    def findKneighbors_fit(self, database):
        self.nNeighbors = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(database)        
    
    def transform(self, sample_df):
        
        #transform
        #onehot encoder
        ocean_labels=self.sample_df['Ocean proximity'].values.reshape(-1, 1)
        onehot_ocean_labels = self.onehot.transform(ocean_labels)
        
        #concatenate onehot encoded values to dataframe
        dfOneHot = pd.DataFrame(onehot_ocean_labels,
                                columns = ["Ocean_"+str(int(i)) for i in range(onehot_ocean_labels.shape[1])])
        encoded_sample = pd.concat([self.sample_df, dfOneHot], axis=1)
        encoded_sample.drop('Ocean proximity', axis = 1, inplace = True) #drop ocean_proximity original column
        
        
        encoded_sample[self.columns] = self.scaler.transform(encoded_sample[self.columns]).astype(float)
        
        return encoded_sample
    
    def findKneighbors(self,sample, n_kneighbors = 5):
        self.housing_ = self.housing.copy(deep = True)
        n_none = [i for i in range(len(sample)) if sample[i] == None]
        
        # Transform list to dataframe
        columns_ = ['Longitude', 'Latitude', 'Age', 'Total rooms', 'Total bedrooms',
                    'Population', 'Households', 'Income', 'Price', 'Ocean proximity']
        
        for index in sorted(n_none, reverse=True):
            del columns_[index]
            del sample[index]
        
        # transform list to dataframe
        self.sample_df = pd.DataFrame(columns = columns_)
        self.sample_df.loc[0] = sample
        
        if len(n_none) != 0:
            #self.sample_df.drop(self.sample_df.columns[n_none],axis=1,inplace=True)
            self.housing_.drop(self.housing_.columns[n_none],axis=1, inplace=True)
        
        self.housing_encoded = self.encode_fit_transform(self.housing_)
        self.findKneighbors_fit(self.housing_encoded)
        
        self.encoded_sample = self.transform(self.sample_df)
        dist, ind = self.nNeighbors.kneighbors(self.encoded_sample, n_kneighbors)
        
        return self.housing.loc[ind[0].tolist()]

In [23]:
house1 = Housing()

In [24]:
returned = house1.findKneighbors([-125, 39, None, None, None, None,
                                  None, None, None, 'NEAR BAY'], 5)
returned

,Longitude,Latitude,Age,Total rooms,Total bedrooms,Population,Households,Income,Price,Ocean proximity
9277,-122.58,38.12,13.0,1.0,1.0,1912.0,770.0,6160.750,604000.0,NEAR BAY
9282,-122.58,38.10,22.0,1.0,1.0,5600.0,2200.0,5807.875,537600.0,NEAR BAY
9274,-122.57,38.11,24.0,1.0,1.0,1583.0,682.0,3997.625,415600.0,NEAR BAY
9275,-122.57,38.11,32.0,1.0,1.0,1706.0,723.0,4338.125,442200.0,NEAR BAY
9283,-122.58,38.08,27.0,1.0,1.0,4406.0,1623.0,7018.750,556200.0,NEAR BAY


# Create new DataFrame

In [7]:
housing = pd.read_csv('housing.csv',sep=',',decimal='.',encoding='utf-8')
new_housing = housing.copy(deep = True)
new_housing['random1'] = np.random.randint(0,5,size=(len(new_housing),1))
new_housing['random2'] = np.random.randint(0,5,size=(len(new_housing),1))

new_housing['total_rooms'] = new_housing['total_rooms']/(new_housing['households']*(new_housing['population']/2.5)) + new_housing['random1']
new_housing['total_bedrooms'] = new_housing['total_bedrooms']/(new_housing['households']*(new_housing['population']/4)) + new_housing['random2']

new_housing['total_rooms'] = new_housing['total_rooms'].apply(np.ceil)
new_housing['total_bedrooms'] = new_housing['total_bedrooms'].apply(np.ceil)
new_housing['median_house_value'] = new_housing['median_house_value']*2 - 15000

new_housing['median_income'] = new_housing['median_income']*1250

new_housing.rename(index=str, columns={"longitude": "Longitude",
                              "latitude": "Latitude",
                              "housing_median_age": "Age",
                              "total_rooms": "Total rooms",
                              "total_bedrooms": "Total bedrooms",
                              "population":"Population",
                              "households":"Households",
                              "median_income":"Income",
                              "median_house_value":"Price",
                              "ocean_proximity":"Ocean proximity"                              
                             }, inplace = True)

In [31]:
new_housing['random'] = np.random.randint(0,10,size=(len(new_housing),1))

In [8]:
new_housing.columns

Index(['Longitude', 'Latitude', 'Age', 'Total rooms', 'Total bedrooms',
       'Population', 'Households', 'Income', 'Price', 'Ocean proximity'],
      dtype='object')

#### Export to csv

In [10]:
export_csv = new_housing.to_csv (r'C:\Users\French -- in English\Documents\SistemasDeRecomendação\California_housing\California_housing\housing\new_housing.csv', index = None, header=True)